# Grid extrapolation of low-carbon generation - sensitivity analysis

## Imports

In [1]:
import math
import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import subprocess
from copy import deepcopy


In [2]:
# Add parent dir to the path, so we can load the module

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))


In [3]:
from energy_insights.region import *
from energy_insights.hourly_data_extrapolator import HourlyDataExtrapolator
from energy_insights.sensitivity_grid_plot import SensitivityGridPlot
from energy_insights.params_library.factors import fill_factors_separate, get_factors_separate
from energy_insights.params_utils import merge_config_into_scenario
from energy_insights.plot_strings import get_grid_strings
from energy_insights.plot_utils import get_scenario_out_file


## Config

In [4]:
runs = [
    {
        'config': {
            'countries': [GERMANY, CZECHIA],
            'year': 2021,
            "analysis_name": "grid-sensitivity",
            'factors': {
                'load_summer': 1.15, 'load_winter': 1.55,
                'generation_solar': 3.58, 'generation_onshore': 2, 'generation_offshore': 3.9,
                'generation_hydro': 1,'generation_nuclear': 0.33},
        },
        'scenarios': [
            {
                'name': 'solar',
                'sensitivity': {
                    'group_name': 'factors',
                    'param_name': 'generation_solar',
                    'values': [1, 2, 3, 4, 5],
                    'bar_width': 0.3,
                }
            },
            {
                'name': 'onshore',
                'sensitivity': {
                    'group_name': 'factors',
                    'param_name': 'generation_onshore',
                    'values': [1, 1.5, 2, 2.5, 3],
                    'bar_width': 0.15,
                }
            },
            {
                'name': 'offshore',
                'sensitivity': {
                    'group_name': 'factors',
                    'param_name': 'generation_offshore',
                    'values': [2, 3, 4, 5, 6],
                    'bar_width': 0.3,
                }
            },
            {
                'name': 'nuclear',
                'sensitivity': {
                    'group_name': 'factors',
                    'param_name': 'generation_nuclear',
                    'values': [0, 0.33, 0.5, 0.66, 1],
                    'bar_width': 0.1,
                }
            },
        ]
    },
]


## Code

In [5]:
for run_number, run in enumerate(runs):
    config = run['config']
    print(f"\nRun number #{run_number}: {config['analysis_name']}")

    for scenario in run['scenarios']:
        print(f"\nRunning scenario ‘{scenario['name']}’")

        params = merge_config_into_scenario(config, scenario)
        # Expand params from a library
        if 'factors_separate' in params:
            params['factors_separate'] = get_factors_separate(params['factors_separate'])
        else:
            params['factors_separate'] = fill_factors_separate(
                params['factors'], params['countries'])

        title, subtitle, factors_str, name = get_grid_strings(params, params['factors_separate'])
        out_filename = get_scenario_out_file(params["name"], params.get("analysis_name", None))

        # Run the optimization for each sensitivity value
        sensitivity = params['sensitivity']
        group_name = sensitivity['group_name']
        param_name = sensitivity['param_name']
        df_list = []
        params_list = []
        for value in sensitivity['values']:
            modified_params = deepcopy(params)
            modified_params[group_name][param_name] = value

            extrapolator = HourlyDataExtrapolator(data_path='../data')
            data = extrapolator.extrapolate_hourly_data_sum(
                modified_params['factors_separate'], params['year'])
            df_list.append(data)
            params_list.append(modified_params)

        plot = SensitivityGridPlot(sensitivity, params_list, df_list,
                                   title, subtitle, factors_str, out_filename, name)
        plot.print_graph()


Run number #0
Running scenario ‘solar’
Plotting sensitivity graph... Done
Running scenario ‘onshore’
Plotting sensitivity graph... Done
Running scenario ‘offshore’
Plotting sensitivity graph... Done
Running scenario ‘nuclear’
Plotting sensitivity graph... Done


<Figure size 1296x864 with 0 Axes>